In [1]:
import os
import tweepy

def credentials_dict() -> dict:
    return {
        "consumer_key": os.getenv("twitter_consumer_api_key"),
        "consumer_secret": os.getenv("twitter_consumer_api_secret"),
        "access_token": os.getenv("twitter_access_api_key"),
        "access_token_secret": os.getenv("twitter_access_api_secret")
    }
credentials = credentials_dict()

In [2]:
auth = tweepy.OAuthHandler(credentials["consumer_key"], credentials["consumer_secret"])
auth.set_access_token(credentials["access_token"], credentials["access_token_secret"])
api = tweepy.API(auth)
available_loc = api.available_trends()

In [3]:
available_loc = api.available_trends()

In [4]:
streaming_client = tweepy.StreamingClient(os.environ.get("bearer_token"))
bb = streaming_client.sample()

In [5]:
streaming_client.sample()
